In [9]:
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup


# driver = webdriver.Chrome(ChromeDriverManager().install())


url = "https://www.hltv.org/stats"
players = []
teams = []
fields = ['Player', 'Team', 'Maps', 'K-D_Diff', 'K/D', 'Rating_(1.0)'] 
fields2 = ['Teams' ,'Maps', 'Won-Lost', 'Pistol_Win_%', 'Round_2_Conversion', 'Round_2_Break'] 

# name of csv file 
def writeToCSV(players):
    filename = "data.csv"

    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(players)


def writeToCSV2(teams):
    filename = "data2.csv"

    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields2) 

        # writing the data rows 
        csvwriter.writerows(teams)
    
    
def get_parsed_page(url):
    # This fixes a blocked by cloudflare error i've encountered
    headers = {
        "referer": "https://www.hltv.org/stats",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    return BeautifulSoup(requests.get(url, headers=headers).text, "lxml")



def driving():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://www.hltv.org/events/5603/blast-premier-spring-showdown-2021')
    element = driver.find_element_by_class_name('team-name')
    print(element)    
        
def playerPerformanceByDate():
    startDate = input("Start date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    endDate = input("End date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    print("Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.")
    mapName = input("Choose the map: ")
    
    dateConversion = url + "/players?startDate=" + startDate + "&endDate=" + endDate + "&maps=de_"+mapName.lower() + "&rankingFilter=Top30";
    print(dateConversion)
    soup = get_parsed_page(dateConversion)
    statTable = soup.tbody;
    for tr in statTable.find_all("tr"):
        newArray = []
        for td in tr.find_all('td'):
            newArray.append(td.get_text())
        players.append(newArray)
    writeToCSV(players)
   

def pistolRoundByDate():
    startDate = input("Start date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    endDate = input("End date (yyyy-mm-dd, leave blank if you want 'All-time' results): ")
    print("Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.")
    mapName = input("Choose the map: ")
    teamSide = input("Enter team ('TERRORIST or COUNTER_TERRORIST'):")
    pistol = url + "/teams/pistols?startDate=" + startDate + "&endDate=" + endDate + "&maps=de_"+mapName.lower()+"&side=" + teamSide.upper() + "&rankingFilter=Top30"; 
    print(pistol)
    soup = get_parsed_page(pistol)
    pistolTable = soup.tbody;

    for tr in pistolTable.find_all("tr"):
        newArray = []
        for td in tr.find_all('td'):
            newArray.append(td.get_text())
        teams.append(newArray)
    writeToCSV2(teams)

    
# Calling functions and creating our csv files dynamically    
playerPerformanceByDate()
pistolRoundByDate()
print('--------------------------------------------------------------------------------')
print('RUN THE 2 CELLS BELOW TO SEE THE "PLAYER DATAFRAME" AND "TEAM PISTOL DATAFRAME"')
print('--------------------------------------------------------------------------------')
    

Start date (yyyy-mm-dd, leave blank for 'All-time' results): 
End date (yyyy-mm-dd, leave blank for 'All-time' results): 
Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.
Choose the map: all
https://www.hltv.org/stats/players?startDate=&endDate=&maps=de_all&rankingFilter=Top30
Start date (yyyy-mm-dd, leave blank for 'All-time' results): 
End date (yyyy-mm-dd, leave blank if you want 'All-time' results): 
Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.
Choose the map: all
Enter team ('TERRORIST or COUNTER_TERRORIST'):terrorist
https://www.hltv.org/stats/teams/pistols?startDate=&endDate=&maps=de_all&side=TERRORIST&rankingFilter=Top30
--------------------------------------------------------------------------------
RUN THE 2 CELLS BELOW TO SEE THE "PLAYER DATAFRAME" AND "TEAM PISTOL DATAFRAME"
------------------------------------------

In [10]:
#Player Performance dataframe
print("-------------------------------")
print('Player Dataframe')
print("-------------------------------")
player_df = pd.read_csv('data.csv')
player_df = player_df.dropna(axis='columns', how ='all')
player_df.head(10)


-------------------------------
Player Dataframe
-------------------------------


,Player,Maps,K-D_Diff,K/D,Rating_(1.0)
0,s1mple,936,5514,1.35,1.25
1,device,1180,4860,1.25,1.16
2,NiKo,1140,3666,1.18,1.15
3,coldzera,1080,4087,1.23,1.14
4,XANTARES,818,1898,1.13,1.11
5,oskar,803,2156,1.16,1.11
6,woxic,582,1653,1.17,1.11
7,Jame,569,2088,1.24,1.10
8,ropz,830,2446,1.18,1.10
9,Brehze,694,1491,1.12,1.09


In [11]:
# Team Pistol dataframe
print("-------------------------------")
print('Team Pistol Dataframe')
print("-------------------------------")
team_pistol_df = pd.read_csv('data2.csv')
team_pistol_df.head(10)

-------------------------------
Team Pistol Dataframe
-------------------------------


,Teams,Maps,Won-Lost,Pistol_Win_%,Round_2_Conversion,Round_2_Break
0,Astralis,1064,529 - 535,49.7%,82.7%,21.9%
1,SK,600,289 - 311,48.2%,80.5%,23.5%
2,Natus Vincere,1027,495 - 532,48.2%,79.0%,22.6%
3,FaZe,1038,520 - 518,50.1%,79.4%,20.3%
4,Vitality,482,252 - 230,52.3%,73.0%,20.0%
5,Liquid,1099,563 - 536,51.2%,82.9%,19.3%
6,Space Soldiers,400,203 - 197,50.7%,84.2%,12.2%
7,mousesports,1134,571 - 563,50.4%,79.9%,22.7%
8,G2,1109,521 - 588,47.0%,81.5%,16.5%
9,fnatic,1093,536 - 557,49.0%,78.5%,20.9%


In [12]:
# url = 'https://www.hltv.org/events/5553/esl-pro-league-season-13'
# def eventDetails():
#     soup = get_parsed_page(url)
#     groups = soup.find_all("div", {"class": "group"})
#     for group in groups:
#         for tr in group.find_all("tr"):
#             print(tr.text)
# eventDetails()